# Appendix 1: Using spin symmetry to optimize the calculations

Author: Gediminas Kiršanskas

In this Appendix we show how numerical calculations can be optimized by using spin symmetry in the system (if it is present).

In [1]:
# Prerequisites
from __future__ import division, print_function
import numpy as np
import qmeq

## Convention

In order to be able to use spin symmetry in the problem the single particle Hamiltonian, tunneling amplitudes, and the lead parameters need to be set up in a particular way. Let us say we have $n=2m$ quantum dot single particle states counting with spin. Then we use the convention that the states with spin up have the labels $0\ldots m-1$ and the states with spin down have the labels $m\ldots n-1$. Also there should be no coupling between the up and down states to get correct results.

Additionally, when the spin degeneracy is present and we want to have a number $m_{\alpha}$ of physical leads, then we need to specify parameters for $n_{\alpha}=2m_{\alpha}$ number of leads. For example, if there are source and drain leads with chemical potentials $\mu_{L}=V/2$ and $\mu_{R}=-V/2$, then we need to specify the following list for chemical potentials:

In [2]:
vbias = 1.0
        #L,up      R,up     L,down    R,down
mulst = [vbias/2, -vbias/2, vbias/2, -vbias/2]

## Example system
Random single particle Hamiltonian with SU(2) spin symmetry:

In [3]:
# Number of single particle states counting with spin
nsingle = 4

vgate = 30.3
# Spin polarized Hamiltonian
hsingle0 = np.random.rand(nsingle//2, nsingle//2)
hsingle0 = hsingle0+hsingle0.T+vgate*np.eye(nsingle//2)
# Introduce spin
hsingle = np.kron(np.eye(2), hsingle0)

Random inter and intra dot coulomb matrix elements:

In [4]:
uinter = 10
uintra = 30
coulomb = np.zeros((nsingle*(nsingle-1)//2, 5), dtype=int)
ind = 0
for j1 in range(nsingle):
    for j2 in range(j1+1, nsingle):
        if j2 == nsingle//2+j1:
            coulomb[ind] = [j1, j2, j2, j1, uintra]
        else:
            coulomb[ind] = [j1, j2, j2, j1, uinter]
        ind += 1
print(coulomb)

[[ 0  1  1  0 10]
 [ 0  2  2  0 30]
 [ 0  3  3  0 10]
 [ 1  2  2  1 10]
 [ 1  3  3  1 30]
 [ 2  3  3  2 10]]


Lead parameters and random tunneling amplitudes:

In [5]:
nleads = 4
tleads0 = 0.1*np.random.rand(2, nsingle//2)
tleads = np.kron(np.eye(2), tleads0)

vbias = 0.5
temp = 5.0
dband = 100.0
mulst = [vbias/2, -vbias/2, vbias/2, -vbias/2]
tlst  = [temp, temp, temp, temp]

Now we set up three systems where no symmetries are used (*indexing='charge'*), just spin projection symmetry $S_{z}$ (*indexing='sz'*) is used, and a full spin symmetry is used (*indexing='ssq'*):

In [6]:
sys_charge = qmeq.Builder(nsingle, hsingle, coulomb, nleads, tleads, mulst, tlst, dband, indexing='charge', kerntype='1vN')
sys_sz = qmeq.Builder(nsingle, hsingle, coulomb, nleads, tleads, mulst, tlst, dband, indexing='sz', kerntype='1vN')
sys_ssq = qmeq.Builder(nsingle, hsingle, coulomb, nleads, tleads, mulst, tlst, dband, indexing='ssq', kerntype='1vN')

In all three cases we obtain the same current:

In [7]:
sys_charge.solve()
sys_sz.solve()
sys_ssq.solve()
print(sys_charge.current)
print(sys_sz.current)
print(sys_ssq.current)

[  1.16916090e-06  -1.16916090e-06   1.16916090e-06  -1.16916090e-06]
[  1.16916090e-06  -1.16916090e-06   1.16916090e-06  -1.16916090e-06]
[  1.16916090e-06  -1.16916090e-06   1.16916090e-06  -1.16916090e-06]


However, the matrix size of Liouvillian (master equation kernel) is reduced for 'sz' and 'ssq' indexing compared to 'charge' indexing:

In [8]:
print(sys_charge.kern.shape)
print(sys_sz.kern.shape)
print(sys_ssq.kern.shape)

(70, 70)
(36, 36)
(20, 20)


This can be achieved, because a lot of reduced density matrix elements are zero or are related by symmetry:

In [9]:
print(sys_charge.phi0.shape)
print(sys_sz.phi0.shape)
print(sys_ssq.phi0.shape)

(70,)
(36,)
(20,)
